In [3]:
!pip3 install pandas
!pip3 install tensorflow
!pip3 install tensorflow-probability
!pip3 install pybullet
!pip3 install jsonpickle

In [4]:
import tensorflow as tf
import tensorflow_probability as tfp
tfpd = tfp.distributions
import pandas as pd
import numpy as np
import pybullet as p
import time
import pybullet_data
import jsonpickle
import os
from grippers import Suction

pybullet build time: Jul 21 2022 19:48:53


In [5]:
df = pd.read_csv('data_points.txt', sep=" ", header=None)
# df.columns = [c for c in (i+'px',i+'py',i+'pz',i+'dx',i+'dy',i+'dz') for i in range(1,4)]
df.columns = ['1cube','1cuboid','1thin_cuboid','1thinner_cuboid','1px','1py','1pz','1dx','1dy','1dz','2cube','2cuboid','2thin_cuboid','2thinner_cuboid','2px','2py','2pz','2dx','2dy','2dz','3cube','3cuboid','3thin_cuboid','3thinner_cuboid','3px','3py','3pz','3dx','3dy','3dz','NaN']
df.drop(df.columns[-1], axis=1, inplace=True)
pd.set_option('display.max_columns', None)

df.head()

,1cube,1cuboid,1thin_cuboid,1thinner_cuboid,1px,1py,1pz,1dx,1dy,1dz,2cube,2cuboid,2thin_cuboid,2thinner_cuboid,2px,2py,2pz,2dx,2dy,2dz,3cube,3cuboid,3thin_cuboid,3thinner_cuboid,3px,3py,3pz,3dx,3dy,3dz
0,1.0,0.0,0.0,0.0,0.516017,0.005864,0.101467,0.03,0.030,0.03,0.0,0.0,0.0,1.0,0.486154,-0.011845,0.086978,0.03,0.030,0.01,1.0,0.0,0.0,0.0,0.504393,0.016517,0.088899,0.03,0.030,0.03
1,1.0,0.0,0.0,0.0,0.512485,0.000377,0.097769,0.03,0.030,0.03,1.0,0.0,0.0,0.0,0.508266,0.012285,0.081395,0.03,0.030,0.03,1.0,0.0,0.0,0.0,0.502419,0.013539,0.084958,0.03,0.030,0.03
2,0.0,0.0,1.0,0.0,0.487709,-0.016338,0.094075,0.06,0.025,0.01,0.0,0.0,1.0,0.0,0.498382,0.007402,0.089118,0.06,0.025,0.01,0.0,0.0,0.0,1.0,0.483544,-0.002319,0.106194,0.03,0.030,0.01
3,1.0,0.0,0.0,0.0,0.501543,0.015722,0.102428,0.03,0.030,0.03,1.0,0.0,0.0,0.0,0.511461,-0.013617,0.103282,0.03,0.030,0.03,0.0,0.0,1.0,0.0,0.510984,-0.004826,0.106737,0.06,0.025,0.01
4,0.0,0.0,0.0,1.0,0.513928,-0.005884,0.083713,0.03,0.030,0.01,0.0,0.0,0.0,1.0,0.500029,-0.014089,0.119384,0.03,0.030,0.01,0.0,0.0,1.0,0.0,0.508541,-0.012000,0.114687,0.06,0.025,0.01


In [6]:
df[(df['1cube'] == 1) & (df['2cube'] == 1) & (df['3thin_cuboid'] == 1)]

,1cube,1cuboid,1thin_cuboid,1thinner_cuboid,1px,1py,1pz,1dx,1dy,1dz,2cube,2cuboid,2thin_cuboid,2thinner_cuboid,2px,2py,2pz,2dx,2dy,2dz,3cube,3cuboid,3thin_cuboid,3thinner_cuboid,3px,3py,3pz,3dx,3dy,3dz
3,1.0,0.0,0.0,0.0,0.501543,0.015722,0.102428,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.511461,-0.013617,0.103282,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.510984,-0.004826,0.106737,0.06,0.025,0.01
8,1.0,0.0,0.0,0.0,0.496316,0.008046,0.091590,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.490922,-0.019487,0.092987,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.499337,-0.010681,0.116579,0.06,0.025,0.01
28,1.0,0.0,0.0,0.0,0.519460,0.008673,0.109805,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.509335,0.012895,0.100932,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.507828,0.007900,0.108386,0.06,0.025,0.01
237,1.0,0.0,0.0,0.0,0.518427,-0.015535,0.083316,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.480148,-0.010756,0.091677,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.496761,-0.007474,0.100945,0.06,0.025,0.01
306,1.0,0.0,0.0,0.0,0.495533,0.018224,0.114009,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.496236,-0.018075,0.115174,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.486459,-0.006864,0.108140,0.06,0.025,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249,1.0,0.0,0.0,0.0,0.481948,0.016685,0.112026,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.513471,-0.006593,0.103219,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.508685,0.012474,0.114000,0.06,0.025,0.01
4326,1.0,0.0,0.0,0.0,0.519925,-0.005678,0.109815,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.493757,-0.012276,0.109580,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.508748,-0.015142,0.092858,0.06,0.025,0.01
4355,1.0,0.0,0.0,0.0,0.518962,0.016762,0.105171,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.489309,-0.019734,0.119133,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.508601,0.001703,0.105090,0.06,0.025,0.01
4361,1.0,0.0,0.0,0.0,0.510126,-0.010022,0.107235,0.03,0.03,0.03,1.0,0.0,0.0,0.0,0.503904,-0.003617,0.088609,0.03,0.03,0.03,0.0,0.0,1.0,0.0,0.517620,-0.002648,0.093889,0.06,0.025,0.01


In [7]:
# with open('data_setup.txt', 'r') as f:
#     for i, line in enumerate(f.readlines()):
#         print('line ', i, line)
#         if i == 1:
#             print(jsonpickle.decode(line))
#             print('it is now line 1')
#         else:
#             break
l1 = '''
[{"py/reduce": [{"py/function": "numpy.core.multiarray._reconstruct"}, {"py/tuple": [{"py/type": "numpy.ndarray"}, {"py/tuple": [0]}, {"py/b64": "Yg=="}]}, {"py/tuple": [1, {"py/tuple": [3]}, {"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["O8", false, true]}, {"py/tuple": [3, "|", null, null, null, -1, -1, 63]}]}, false, [{"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["U4", false, true]}, {"py/tuple": [3, "<", null, null, null, 16, 4, 8]}]}, {"py/b64": "YwAAAHUAAABiAAAAZQAAAA=="}]}]}, {"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["U4", false, true]}, {"py/tuple": [3, "<", null, null, null, 16, 4, 8]}]}, {"py/b64": "YwAAAHUAAABiAAAAZQAAAA=="}]}]}, {"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["U11", false, true]}, {"py/tuple": [3, "<", null, null, null, 44, 4, 8]}]}, {"py/b64": "dAAAAGgAAABpAAAAbgAAAF8AAABjAAAAdQAAAGIAAABvAAAAaQAAAGQAAAA="}]}]}]]}]}, {"py/reduce": [{"py/function": "numpy.core.multiarray._reconstruct"}, {"py/tuple": [{"py/type": "numpy.ndarray"}, {"py/tuple": [0]}, {"py/b64": "Yg=="}]}, {"py/tuple": [1, {"py/tuple": [3, 3]}, {"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["f8", false, true]}, {"py/tuple": [3, "<", null, null, null, -1, -1, 0]}]}, false, {"py/b64": "rFlKqKMM4D+dPw8+jRmQP0mCYU68OLo/gHFc4eNd4D+mG1rT/uKLv3mBkkuvcLo/xLXIpfpZ4D8uctKNksRzvy7bllglU7s/"}]}]}]
'''
t1 = jsonpickle.decode(l1)
t1

[array(['cube', 'cube', 'thin_cuboid'], dtype=object),
 array([[ 0.50154288,  0.01572247,  0.10242822],
        [ 0.5114612 , -0.01361655,  0.10328193],
        [ 0.51098378, -0.00482614,  0.10673746]])]

l1 = '''
[{"py/reduce": [{"py/function": "numpy.core.multiarray._reconstruct"}, {"py/tuple": [{"py/type": "numpy.ndarray"}, {"py/tuple": [0]}, {"py/b64": "Yg=="}]}, {"py/tuple": [1, {"py/tuple": [3]}, {"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["O8", false, true]}, {"py/tuple": [3, "|", null, null, null, -1, -1, 63]}]}, false, [{"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["U4", false, true]}, {"py/tuple": [3, "<", null, null, null, 16, 4, 8]}]}, {"py/b64": "YwAAAHUAAABiAAAAZQAAAA=="}]}]}, {"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["U4", false, true]}, {"py/tuple": [3, "<", null, null, null, 16, 4, 8]}]}, {"py/b64": "YwAAAHUAAABiAAAAZQAAAA=="}]}]}, {"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["U11", false, true]}, {"py/tuple": [3, "<", null, null, null, 44, 4, 8]}]}, {"py/b64": "dAAAAGgAAABpAAAAbgAAAF8AAABjAAAAdQAAAGIAAABvAAAAaQAAAGQAAAA="}]}]}]]}]}, {"py/reduce": [{"py/function": "numpy.core.multiarray._reconstruct"}, {"py/tuple": [{"py/type": "numpy.ndarray"}, {"py/tuple": [0]}, {"py/b64": "Yg=="}]}, {"py/tuple": [1, {"py/tuple": [3, 3]}, {"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["f8", false, true]}, {"py/tuple": [3, "<", null, null, null, -1, -1, 0]}]}, false, {"py/b64": "rFlKqKMM4D+dPw8+jRmQP0mCYU68OLo/gHFc4eNd4D+mG1rT/uKLv3mBkkuvcLo/xLXIpfpZ4D8uctKNksRzvy7bllglU7s/"}]}]}]
'''
t1 = jsonpickle.decode(l1)

######

ASSETS_ROOT = os.path.abspath("urdf")
obj_ids = {'fixed': [], 'rigid': [], 'deformable': []}
homej = np.array([-1, -0.5, 0.5, -0.5, -0.5, 0]) * np.pi

physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-9.81)

target = (-0.07796166092157364, 0.005451506469398737, -0.06238798052072525)
dist = 1.0
yaw = 89.6000747680664
pitch = -17.800016403198242
p.resetDebugVisualizerCamera(dist,yaw,pitch,target)

planeId = p.loadURDF(os.path.join(ASSETS_ROOT, 'plane/plane.urdf'))

origin = np.zeros(3)
noRotation = p.getQuaternionFromEuler([0,0,0])

ur5 = p.loadURDF(os.path.join(ASSETS_ROOT, 'ur5/ur5.urdf'), origin, noRotation, 
                useFixedBase=True)
suc = Suction(ASSETS_ROOT, ur5, 9, obj_ids)
n_joints = p.getNumJoints(ur5)
joints = [p.getJointInfo(ur5, i) for i in range(n_joints)]
joints = [j[0] for j in joints if j[2] == p.JOINT_REVOLUTE]

for j, hj in zip(joints, homej):
    p.resetJointState(ur5, j, hj)

# only static dimension shapes
shapes = ['cube', 'cuboid', 'thin_cuboid', 'thinner_cuboid']
valid_rots = {
    'cube': ([0], [0], [0]),
    'cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
    'thin_cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
    'thinner_cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
}

dims = {
    'cube': np.array([0.03, 0.03, 0.03]),
    'cuboid': np.array([0.03, 0.025, 0.06]),
    'thin_cuboid': np.array([0.06, 0.025, 0.01]),
    'thinner_cuboid': np.array([0.03, 0.03, 0.01])
}

shape_oh = {
    'cube': [1, 0, 0, 0],
    'cuboid': [0, 1, 0, 0],
    'thin_cuboid': [0, 0, 1, 0],
    'thinner_cuboid': [0, 0, 0, 1],
}

obj_num = 3

objPos = np.array([0.5,0,0.1])

# t1[0][0] = 'cube'
# t1[0][1] = 'c3'
# t1[1][1] += np.array([0,0,0]) 
# t1[0][2] = 'thinner_cuboid'

for i in range(3):
    add = np.array([0,0,0])
    ot = t1[0][i]
    if i == 2:
        add = np.array([+0.003,+0.007,+0.006])
    o1 = p.loadURDF(os.path.join(ASSETS_ROOT, f'objects/{ot}.urdf'), t1[1][i]+add, noRotation)
    for i in range (150):
        p.stepSimulation()
        time.sleep(1./240.)
        

while True:
    p.stepSimulation()
    time.sleep(1./240.)
    print()

p.disconnect()

In [ ]:
######

ASSETS_ROOT = os.path.abspath("urdf")
obj_ids = {'fixed': [], 'rigid': [], 'deformable': []}
homej = np.array([-1, -0.5, 0.5, -0.5, -0.5, 0]) * np.pi

physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-9.81)

target = (-0.07796166092157364, 0.005451506469398737, -0.06238798052072525)
dist = 1.0
yaw = 89.6000747680664
pitch = -17.800016403198242
p.resetDebugVisualizerCamera(dist,yaw,pitch,target)

planeId = p.loadURDF(os.path.join(ASSETS_ROOT, 'plane/plane.urdf'))

origin = np.zeros(3)
noRotation = p.getQuaternionFromEuler([0,0,0])

ur5 = p.loadURDF(os.path.join(ASSETS_ROOT, 'ur5/ur5.urdf'), origin, noRotation, 
                useFixedBase=True)
suc = Suction(ASSETS_ROOT, ur5, 9, obj_ids)
n_joints = p.getNumJoints(ur5)
joints = [p.getJointInfo(ur5, i) for i in range(n_joints)]
joints = [j[0] for j in joints if j[2] == p.JOINT_REVOLUTE]

for j, hj in zip(joints, homej):
    p.resetJointState(ur5, j, hj)

# only static dimension shapes
shapes = ['cube', 'cuboid', 'thin_cuboid', 'thinner_cuboid']
valid_rots = {
    'cube': ([0], [0], [0]),
    'cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
    'thin_cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
    'thinner_cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
}

def load_obj(otype):
    return p.loadURDF(os.path.join(ASSETS_ROOT, f'objects/{otype}.urdf'), origin, noRotation)

objPos = np.array([0.5,0,0.1])

objs = [0]*8
objs[0] = load_obj('cuboid')
objs[1] = load_obj('cuboid')
objs[2] = load_obj('thin_cuboid')
objs[3] = load_obj('cube')
objs[4] = load_obj('cube')
objs[5] = load_obj('cuboid')
objs[6] = load_obj('cube')
objs[7] = load_obj('cuboid')
        

while True:
    p.stepSimulation()
    time.sleep(1./240.)
    for i, ob in enumerate(objs):
        print(f'o{i}: {p.getBasePositionAndOrientation(ob)}')
    print()

p.disconnect()

o0: ((9.73414551506065e-05, -0.004737054687441361, 0.00234887024255328), (0.0006576997654864156, -0.00035280186472103563, -0.0035249899142229973, 0.999993508682915))
o1: ((0.00022636745570656382, -0.0028465465891736826, 0.002424030036970943), (6.246342806543516e-05, -0.00010003504893323758, -0.002268398980647423, 0.9999974202253592))
o2: ((0.0002604761188791727, -0.0001912219333533751, 0.0025952677594255195), (0.01293214836082679, -0.0031174708488125305, -0.006148597290217865, 0.9998926120667375))
o3: ((-0.003812447009524352, -0.00012303507831518484, 0.003739803277401859), (-0.00023798731858669757, -0.00023061714938250993, 0.0005865050122357804, 0.9999997730947928))
o4: ((-0.0005134224356963009, 0.0004882520032254423, 0.0035985505874925293), (0.0006481528385346042, 0.00499801078610233, 0.023257457040863953, 0.9997168051393716))
o5: ((-0.00012422192339842653, 0.002183354422388704, 0.0030170068353839665), (0.0047457007654285455, -0.018382422401698205, -0.023314777052795777, 0.99954789081

o0: ((-0.001886830711456693, -0.15331216646559578, 0.012542932444286706), (0.740700577905374, -0.024653278070904488, -0.021627593985233517, 0.671034363463909))
o1: ((-0.0007282968831664351, -0.0853980438413127, 0.02032964120963067), (0.595459662286077, -0.017809990629210175, -0.023804665799986857, 0.802834934908845))
o2: ((0.00394698135205184, -0.002616561191157974, 0.004964325898031242), (-0.00021189038171283523, -0.00018064567725256778, -0.026161559204371198, 0.9996576890564097))
o3: ((-0.1549844822403444, -0.007872985449184714, 0.015270906754756406), (0.012245808371695525, -0.001766905000674109, 0.006787585733650995, 0.9999004184937371))
o4: ((0.010901612935649525, -0.038226572009024816, 0.033038910764637865), (-0.20712750679478104, 0.02481595609359924, -0.004638217716587573, 0.977988165157714))
o5: ((-0.017912585584718435, 0.1810225010952045, 0.02718448013555272), (0.20089060383541063, -0.7854643071623804, -0.14405541975210104, 0.5673947686616538))
o6: ((-0.018678014237269062, 0.05

o0: ((-0.0017839179860550485, -0.15189428318195822, 0.012417454897102002), (0.7065426977438573, -0.023118618867184186, -0.02310682523998676, 0.706915143672729))
o1: ((-0.0010167439751263333, -0.09056119280993132, 0.012443926676711984), (0.7072150147654371, -0.02098838931445201, -0.02096729269584856, 0.7063758086467247))
o2: ((0.003920689865719626, -0.0021860752871750598, 0.004989675798343735), (-1.5224479405967949e-05, 9.614695204303825e-07, -0.02866057563186002, 0.9995892012080468))
o3: ((-0.15482628596614825, -0.007751248153711836, 0.014981450493818148), (-0.00023556049149720375, 4.744485762999274e-05, 0.000659800475272971, 0.9999997534617562))
o4: ((0.010183693990816249, -0.03264665980157245, 0.014441358796457023), (-0.6977171424741725, 0.019362859660820037, -0.020386776056143516, 0.7158213800421634))
o5: ((-0.019584011586588054, 0.21615079330710363, 0.01914253466031642), (-0.08988710287613429, -0.7009221344642287, 0.4821378137361202, 0.5178528736237731))
o6: ((-0.011580346807169184

o0: ((-0.001783516300314929, -0.15190082114752626, 0.012488212531986129), (0.7067271298258352, -0.02310017794980012, -0.023125463446950326, 0.7067307540267287))
o1: ((-0.0010155050128458456, -0.09054668730318748, 0.012488991863897883), (0.7068022771593975, -0.02096489455877366, -0.02099016702942845, 0.7067888136399064))
o2: ((0.003920700700942973, -0.002186129234669595, 0.004989742415756258), (-9.824752585431993e-06, 2.2639818954784986e-06, -0.02866059005750427, 0.9995892008599855))
o3: ((-0.1548272291423983, -0.007758488858082345, 0.01498928807410489), (7.159801631114848e-06, 2.493794645056338e-05, 0.0006582622605058238, 0.9999997830087927))
o4: ((0.01021804809549688, -0.03225440498772412, 0.01498577002092984), (-0.706795267378874, 0.019893124704794662, -0.01986645575164436, 0.7068592770390536))
o5: ((-0.026357465344581364, 0.225553131095785, 0.012487130091949855), (0.32290297141293917, 0.6290759528202303, -0.6290099859851107, -0.3230225288846349))
o6: ((-0.01208178241796644, 0.048510

o0: ((-0.0017839774312948592, -0.1519009245407992, 0.012489882563856336), (0.7067305022757101, -0.023112846892350002, -0.023112840921619453, 0.7067273802861931))
o1: ((-0.0010157877157184955, -0.09054641937376424, 0.012489827570950346), (0.7067948110560874, -0.020973016140515346, -0.020981972609000355, 0.7067962821659398))
o2: ((0.00392069873013402, -0.0021861894994296546, 0.0049898378979968405), (-3.7178982941706525e-06, 2.343943185410102e-06, -0.028660614872767734, 0.9995892001896569))
o3: ((-0.15482805520434706, -0.007758631267871435, 0.014989305513423614), (1.4869157210815585e-05, 1.6282898678742165e-05, 0.0006551495737771952, 0.9999997851463827))
o4: ((0.010217718561782305, -0.03225294498311535, 0.014989814602415153), (-0.7068295870072702, 0.019884599989531956, -0.019875000846689823, 0.7068249584980232))
o5: ((-0.02635853771957995, 0.2255547919327062, 0.012489677860847224), (0.32296323609486716, 0.6290463918598778, -0.6290383661144695, -0.32296457851479426))
o6: ((-0.0120743410650

...

o0: ((-0.0017840010564168897, -0.15190091531256522, 0.012489902993180589), (0.70673034097282, -0.0231125603383435, -0.02311328935847507, 0.706727536295339))
o1: ((-0.0010158061088296746, -0.09054644770658517, 0.012489837532003945), (0.7067950041047039, -0.02097305030945918, -0.020981588732619898, 0.7067960994994269))
o2: ((0.003920698951406023, -0.0021862001331159208, 0.00498985139773891), (-2.6012977215140047e-06, 2.4852959504785055e-06, -0.028660635186238723, 0.9995891996104089))
o3: ((-0.1548298865931547, -0.007758364233798373, 0.014989304474285222), (1.5391400669726064e-05, 1.576933608433221e-05, 0.0006453045063293212, 0.9999997915482417))
o4: ((0.010217568145859189, -0.03225279915280773, 0.014989706363328845), (-0.7068323727263469, 0.01988437774356484, -0.019877262654562942, 0.706822115399027))
o5: ((-0.02635715089488141, 0.22555392000666413, 0.012489722368911344), (0.3229697219373665, 0.6290432303494321, -0.6290362819115485, -0.32296830974903173))
o6: ((-0.012073975166563982, 0.0

o0: ((-0.0017840369798900076, -0.15190091166924324, 0.012489895527770637), (0.7067303065387794, -0.02311315332259616, -0.023112770608217356, 0.7067275683018887))
o1: ((-0.0010158096130269483, -0.0905464522865963, 0.01248984504220761), (0.7067948681790888, -0.020972945468632657, -0.020981543013490825, 0.7067962398929747))
o2: ((0.003920697339710251, -0.0021862066260973255, 0.004989863129282365), (-1.919582577852555e-06, 2.3889055890289346e-06, -0.028660645717671223, 0.9995891993102239))
o3: ((-0.1548306214931648, -0.0077582491366836905, 0.01498930746824623), (1.4673193415676816e-05, 1.1266100548328229e-05, 0.0006420402332853877, 0.9999997937210343))
o4: ((0.010217520972142515, -0.03225281809748152, 0.014989812534652164), (-0.7068317217516015, 0.019884617556251618, -0.019877796850179156, 0.7068227446133092))
o5: ((-0.02635659727049827, 0.22555352340190327, 0.012489781044537823), (0.3229695137226173, 0.6290434862519618, -0.6290341485239503, -0.3229721746536423))
o6: ((-0.01207390452761472

o0: ((-0.0017840397294806548, -0.15190091132208636, 0.0124898988023562), (0.706730332397902, -0.023112822721806373, -0.02311317029857759, 0.7067275401831531))
o1: ((-0.0010158129003450319, -0.09054646371749817, 0.012489838752325913), (0.7067949357172731, -0.020972833875212617, -0.020981510784123723, 0.7067961766229881))
o2: ((0.003920700169839139, -0.00218619816593149, 0.0049898391643940915), (-2.770960618939156e-06, 2.6518977768129127e-06, -0.02866064664366685, 0.9995891992810128))
o3: ((-0.15483110725712904, -0.007758174262049593, 0.014989304474284069), (1.5391573545059728e-05, 1.5769152707895646e-05, 0.0006386893167962766, 0.9999997957951741))
o4: ((0.010217507589295615, -0.03225277216078383, 0.014989767324896023), (-0.7068325959913266, 0.019885645707192985, -0.019877575397135652, 0.7068218476641066))
o5: ((-0.026356146865419774, 0.22555326485408395, 0.012489725572494038), (0.32297238007807455, 0.6290417794138841, -0.6290341076144993, -0.3229727123398337))
o6: ((-0.01207386855427217

o0: ((-0.0017840550204041052, -0.15190090912019397, 0.012489901146007214), (0.7067303277847311, -0.023112731578015064, -0.02311335185773842, 0.7067275418392788))
o1: ((-0.0010158092853315148, -0.09054647407317015, 0.012489843347098987), (0.7067949444782207, -0.020972505359970493, -0.020981685079642962, 0.7067961724359957))
o2: ((0.003920699427933321, -0.0021861992886123742, 0.004989842013641232), (-2.6325388981622615e-06, 2.6206788837061725e-06, -0.028660655732016614, 0.9995891990208841))
o3: ((-0.15483184215326123, -0.007758059162306658, 0.0149893074682411), (1.4673322591505649e-05, 1.1265952872064738e-05, 0.0006354249757317903, 0.9999997979464157))
o4: ((0.01021750226313004, -0.03225271601644769, 0.014989751515298189), (-0.7068336936969918, 0.019887033096206704, -0.019877096400775635, 0.7068207243763951))
o5: ((-0.026355676978284025, 0.22555296754328005, 0.012489729496885099), (0.32297425529876495, 0.629040796477775, -0.62903334297907, -0.32297424077066933))
o6: ((-0.0120738733620590

o0: ((-0.0017840710289996123, -0.15190090770985248, 0.012489900215598645), (0.7067303340808467, -0.023112798448858447, -0.02311335008520661, 0.7067275334141693))
o1: ((-0.0010158128827733855, -0.09054648045064595, 0.012489833423799796), (0.7067948891636802, -0.020972424355788805, -0.0209816323675707, 0.7067962317188272))
o2: ((0.00392069635161906, -0.002186209146105754, 0.004989869113781039), (-1.6207615158933416e-06, 2.373352906834393e-06, -0.028660662882068328, 0.999589198818645))
o3: ((-0.1548323279151838, -0.007757984289040003, 0.01498930447424313), (1.5391733058793723e-05, 1.576898465495471e-05, 0.0006320739885348714, 0.9999997999984334))
o4: ((0.010217443381535945, -0.03225273994585037, 0.014989757270721665), (-0.70683290350377, 0.019886915340764182, -0.019877953541283973, 0.7068214937915159))
o5: ((-0.02635521146640122, 0.22555267447664637, 0.012489725467769702), (0.3229763000371186, 0.6290397625583661, -0.6290326288124602, -0.3229756006637857))
o6: ((-0.012073798213356376, 0.04

In [ ]:
p.disconnect()

In [8]:
import tensorflow as tf
import tensorflow_probability as tfp
tfpd = tfp.distributions

d = tfpd.JointDistributionSequential([
    tfpd.Uniform(),
    tfpd.Uniform(),
    tfpd.Uniform()
])

print(d.resolve_graph())

(('x2', ()), ('x1', ()), ('x', ()))


2022-11-03 00:45:30.609567: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 00:45:30.633557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 00:45:30.633728: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 00:45:30.634197: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [9]:
p.disconnect()

error: Not connected to physics server.